In [1]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instrument

In [2]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [16,64]
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [3]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [4]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [5]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2020-01-06T13:00:00.000000000Z,1.11983,1.12032,1.11911,1.11976,1.117484,1.117742
1,2020-01-06T14:00:00.000000000Z,1.11978,1.12002,1.11826,1.11832,1.117631,1.117692
2,2020-01-06T15:00:00.000000000Z,1.11836,1.11902,1.11822,1.11888,1.117808,1.117645
3,2020-01-06T16:00:00.000000000Z,1.11890,1.11898,1.11810,1.11872,1.117956,1.117596
4,2020-01-06T17:00:00.000000000Z,1.11870,1.11961,1.11843,1.11952,1.118158,1.117556


In [6]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

In [7]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)

In [8]:
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()
df_trades["DELTA"] = ( df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]

In [9]:
from dateutil.parser import *
df_trades["time"] = [parse(x) for x in df_trades.time]

In [10]:
df_trades["DURATION"] = df_trades.time.diff().shift(-1)

In [11]:
df_trades["DURATION"] = [x.total_seconds() / 3600 for x in df_trades.DURATION]

In [12]:
df_trades[['time', 'DURATION']].tail()

,time,DURATION
18486,2022-12-22 03:00:00+00:00,17.0
18503,2022-12-22 20:00:00+00:00,25.0
18528,2022-12-23 21:00:00+00:00,120.0
18576,2022-12-28 21:00:00+00:00,20.0
18596,2022-12-29 17:00:00+00:00,NaN


In [13]:
df_trades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 373 entries, 2 to 18596
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype                  
---  ------     --------------  -----                  
 0   time       373 non-null    datetime64[ns, tzutc()]
 1   mid_o      373 non-null    float64                
 2   mid_h      373 non-null    float64                
 3   mid_l      373 non-null    float64                
 4   mid_c      373 non-null    float64                
 5   MA_16      373 non-null    float64                
 6   MA_64      373 non-null    float64                
 7   DIFF       373 non-null    float64                
 8   DIFF_PREV  373 non-null    float64                
 9   IS_TRADE   373 non-null    int64                  
 10  DELTA      372 non-null    float64                
 11  GAIN       372 non-null    float64                
 12  DURATION   372 non-null    float64                
dtypes: datetime64[ns, tzutc()](1), float64(11), int6

In [14]:
df_all = pd.read_pickle("all_trades.pkl")

In [15]:
df_all.describe()

,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,MASHORT,MALONG,DURATION
count,66885.000000,6.688500e+04,6.688500e+04,66885.000000,66885.000000,66885.00000,66885.000000,66885.000000,66885.000000
mean,35.593336,-4.510093e-04,4.534397e-04,-0.001181,-0.353739,-4.05211,11.822741,61.336682,49.985169
std,58.472771,3.387540e-02,3.339264e-02,1.000007,93.981241,93.89451,12.499719,64.796491,83.298226
min,0.552120,-8.646146e-01,-7.054375e-01,-1.000000,-945.800000,-1139.80000,4.000000,8.000000,1.000000
25%,0.926530,-2.662500e-04,-2.587500e-04,-1.000000,-30.200000,-39.90000,4.000000,16.000000,6.000000
50%,1.305740,-1.110223e-16,4.687500e-07,-1.000000,0.300000,-14.10000,8.000000,32.000000,17.000000
75%,84.844000,2.643750e-04,2.629167e-04,1.000000,30.900000,10.40000,16.000000,96.000000,64.000000
max,171.560000,6.221250e-01,1.131125e+00,1.000000,1327.900000,1327.90000,64.000000,256.000000,1516.000000


In [16]:
df_trades["GAIN"].sum()

1482.7999999999752

In [17]:
df_plot = df_ma.iloc[100:250].copy()

In [18]:
df_plot.shape

(150, 10)

In [19]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(
        x=df_plot.time,
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True,
)
fig.show()